In [1]:
from sklearn.datasets import fetch_20newsgroups
import numpy as np
newsgroups_train = fetch_20newsgroups(subset = 'train')
newsgroups_text = fetch_20newsgroups(subset = 'test')
newsgroups_all = fetch_20newsgroups(subset = 'all')

In [2]:
from pprint import pprint
pprint(list(newsgroups_train.target_names))

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']


In [3]:
print('texts num in train: %d'%(len(newsgroups_train.data)))
print('texts num in test: %d'%(len(newsgroups_text.data)))

texts num in train: 11314
texts num in test: 7532


In [4]:
print('context',newsgroups_text.data[0])
print('label',newsgroups_text.target[0])

context From: v064mb9k@ubvmsd.cc.buffalo.edu (NEIL B. GANDLER)
Subject: Need info on 88-89 Bonneville
Organization: University at Buffalo
Lines: 10
News-Software: VAX/VMS VNEWS 1.41
Nntp-Posting-Host: ubvmsd.cc.buffalo.edu


 I am a little confused on all of the models of the 88-89 bonnevilles.
I have heard of the LE SE LSE SSE SSEI. Could someone tell me the
differences are far as features or performance. I am also curious to
know what the book value is for prefereably the 89 model. And how much
less than book value can you usually get them for. In other words how
much are they in demand this time of year. I have heard that the mid-spring
early summer is the best time to buy.

			Neil Gandler

label 7


In [6]:
w2v_cropus_file = './corpus/corpus_text8.txt'

def loadword2vec(filename):
    '''
    @param w2v_file: w2v file path
    @return word2vec_dict: a dictionary includes word and corresponding vector{word:vector}
    @return word_dim: dimension of word
    '''
    vocab = []
    embed = []
    fr = open(filename,'r')
    line = fr.readline().strip()
    
    word_dim = int(line.split(' ')[1])
    vocab.append('UNK')
    embed.append([0]*word_dim)
    
    for line in fr:
        row = line.strip().split(' ')
        vocab.append(row[0])
        embed.append(row[1:])
    
    print('load w2v over')
    fr.close()
    return vocab,embed

vocab,embed = loadword2vec(w2v_cropus_file)
vocab_size = len(vocab)
embedding_dim = len(embed[0])


print(embedding_dim)
print(vocab_size)

load w2v over
200
47135


In [7]:
print(len(newsgroups_train.data[0]))

721


In [8]:
def create_lookup_tables(text):
    int_to_vocab = {}
    vocab_to_int = {}
    for word,i in enumerate(text):
        int_to_vocab[i],vocab_to_int[word] = word,i
    
    return int_to_vocab,vocab_to_int




In [26]:
def clear_data(string):
    import re
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower() 

In [31]:
from tflearn.data_utils import pad_sequences

vocabulary_word2index,vocabulary_index2word = create_lookup_tables(vocab) 

X = []
#train  clear->word2index->list append
for i,sentence in enumerate(newsgroups_train.data):
   
    sentence = clear_data(sentence) 
    s = sentence.split(" ")  
    x = [vocabulary_word2index.get(w,vocabulary_word2index['UNK']) for w in s]
    X.append(x)
    


In [32]:
print(len(X[0]))
print(len(X))
print(X[0])

145
11314
[28, 0, 0, 0, 7711, 0, 100, 0, 609, 1798, 0, 576, 154, 983, 11, 33, 0, 0, 0, 12631, 1560, 0, 0, 0, 7711, 709, 127, 2, 4152, 0, 354, 787, 823, 0, 72, 18, 29818, 88, 2615, 108, 60, 160, 0, 746, 25, 33, 983, 72, 798, 1, 43, 137, 27, 18, 6, 0, 2931, 1327, 983, 0, 4163, 7, 32, 28, 1, 329, 0, 128, 0, 27, 18, 89, 6, 40212, 1, 4379, 40, 2269, 221, 5, 536, 0, 1, 927, 35118, 18, 834, 28, 1, 995, 2, 1, 341, 33, 11, 50, 72, 1006, 88, 2615, 53, 0, 6, 518, 121, 0, 1030, 39191, 0, 81, 2, 443, 0, 100, 33, 983, 11, 109, 0, 96, 0, 29, 3537, 3862, 207, 39, 25, 33, 21423, 2398, 983, 0, 3877, 112, 2214, 4414, 0, 2796, 754, 7, 207, 19, 772, 5931, 0]
721


    save&padding

In [34]:
#max sentence length
len_counter = np.zeros(len(X))
for i,w in enumerate(X):
    len_counter[i] = len(w)

print(np.max(len_counter))
print(np.min(len_counter))
print(np.mean(len_counter))
print(np.median(len_counter))
print(np.percentile(len_counter,90))   

21273.0
20.0
360.608538094
209.0
574.0


In [ ]:
padding_length = np.percentile(len_counter,90)
